## 1. Data pre-process

In [77]:
#Import all the necessary packages
import numpy as np
import pandas as pd
import tensorflow as tf
import numpy as np

from tensorflow import keras
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense, SimpleRNN

#### 1). Data Check

In [64]:
#Read the data at first glance, check each identical characters in this file.
data = open('data.txt', 'r').read()
data = data.lower()
chars = list(set(data))
print(sorted(chars))
data_size, vocab_size = len(data), len(chars)
print('There are %d total characters and %d unique characters in the data.' % (data_size, vocab_size))

['\n', ' ', "'", '*', '-', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '`', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']
There are 126071 total characters and 42 unique characters in the data.


In [65]:
#Show the details of the data in two groups.
df = pd.read_csv('data.txt', sep=" ", header=None)
df.columns = ["Input", "Output"]
df.head()
# show(df)

,Input,Output
0,apprehensive,apprehensiveway
1,desperately,esperatelyday
2,grew,ewgray
3,niggardly,iggardlynay
4,brought,oughtbray


#### 2). Set up training details. 

In [66]:
#Set up training parameters
batch_size = 64
epochs = 100
latent_dim = 256
num_samples = 6511

#### 3). Word seperation and add start/end mark. 

In [67]:
input_texts = []
target_texts = []
input_chars = set()
target_chars = set()
with open('data.txt', 'r', encoding='utf-8') as f:
    lines = f.read().split('\n')
#Seperate words into input and target group   
for line in lines:
    input_text, target_text= line.split(' ')
    target_text = '\t' + target_text + '\n'
    input_texts.append(input_text)
    target_texts.append(target_text)
    for char in input_text:
        if char not in input_chars:
            input_chars.add(char)
    for char in target_text:
        if char not in target_chars:
            target_chars.add(char)

In [68]:
#Check the data detail
input_chars = sorted(list(input_chars))
target_chars = sorted(list(target_chars))
num_encoder_tokens = len(input_chars)
num_decoder_tokens = len(target_chars)
max_encoder_seq_length = max([len(i) for i in input_texts])
max_decoder_seq_length = max([len(i) for i in target_texts])

In [69]:
print("Number of samples:", len(input_texts))
print("Number of unique input tokens:", num_encoder_tokens)
print("Number of unique output tokens:", num_decoder_tokens)
print("Max sequence length for inputs:", max_encoder_seq_length)
print("Max sequence length for outputs:", max_decoder_seq_length)

Number of samples: 6511
Number of unique input tokens: 40
Number of unique output tokens: 42
Max sequence length for inputs: 18
Max sequence length for outputs: 24


#### 4). Word embedding and one-hot vector

In [70]:
#Tokenize the words and word embedding
input_token_index = dict(
                    [(char, i) for i, char in enumerate(input_chars)])
target_token_index = dict(
                    [(char, i) for i, char in enumerate(target_chars)])

In [71]:
#Create zero vectors for one-hot vector.
encoder_input_data = np.zeros(
                    (len(input_texts), max_encoder_seq_length, num_encoder_tokens),dtype='float32')
decoder_input_data = np.zeros(
                    (len(target_texts), max_decoder_seq_length, num_decoder_tokens),dtype='float32')
decoder_target_data = np.zeros(
                    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),dtype='float32')

In [72]:
#Check the size of each group data sets.
print(encoder_input_data.shape)
print(decoder_input_data.shape)
print(decoder_target_data.shape)

(6511, 18, 40)
(6511, 24, 42)
(6511, 24, 42)


In [73]:
#Make sure the index for each word is correct.
print('The input index:')
print(input_token_index)
print('\n The target index:')
print(target_token_index)

The input index:
{"'": 0, '*': 1, '-': 2, '0': 3, '1': 4, '2': 5, '3': 6, '4': 7, '5': 8, '6': 9, '7': 10, '8': 11, '9': 12, '`': 13, 'a': 14, 'b': 15, 'c': 16, 'd': 17, 'e': 18, 'f': 19, 'g': 20, 'h': 21, 'i': 22, 'j': 23, 'k': 24, 'l': 25, 'm': 26, 'n': 27, 'o': 28, 'p': 29, 'q': 30, 'r': 31, 's': 32, 't': 33, 'u': 34, 'v': 35, 'w': 36, 'x': 37, 'y': 38, 'z': 39}

 The target index:
{'\t': 0, '\n': 1, "'": 2, '*': 3, '-': 4, '0': 5, '1': 6, '2': 7, '3': 8, '4': 9, '5': 10, '6': 11, '7': 12, '8': 13, '9': 14, '`': 15, 'a': 16, 'b': 17, 'c': 18, 'd': 19, 'e': 20, 'f': 21, 'g': 22, 'h': 23, 'i': 24, 'j': 25, 'k': 26, 'l': 27, 'm': 28, 'n': 29, 'o': 30, 'p': 31, 'q': 32, 'r': 33, 's': 34, 't': 35, 'u': 36, 'v': 37, 'w': 38, 'x': 39, 'y': 40, 'z': 41}


In [78]:
#Create one hot vector for all characters
for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.

    for t, char in enumerate(target_text):
        decoder_input_data[i, t, target_token_index[char]] = 1.
        if t > 0:
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.

## 2.Build our model 

#### 1). Building encoder

In [79]:
#For the encoder we use LSTM to process the data in our RNN model. 
#Here we take the input from our data and output the states, hidden state and conveyor state, 
#and pass it to the decoder

encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)

In [80]:
#Save the state and pass it to the decoder
encoder_states = [state_h, state_c]

In [81]:
#Input the states from the encoder and output the target word combination.
decoder_inputs = Input(shape=(None, num_decoder_tokens))
decoder_rnn = LSTM(latent_dim, return_sequences=True, return_state=True)
#Use the states from encoder as initial_state
decoder_outputs, _, _ = decoder_rnn(decoder_inputs, initial_state=encoder_states)
#Fully connected layer for identification through softmax function
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

## Model training 

In [82]:
#Combine models in Model function
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [99]:
#Train the function with rmsprop regulation, and verify the accuracy with cross-entropy.
#Data is seperated into two groups, 20% as validation(test) set, the rest as training set.
model.compile(optimizer='rmsprop', loss='categorical_crossentropy',metrics=['accuracy'])
hist = model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=batch_size,
          epochs=epochs,
          validation_split=0.2)

Epoch 1/100
82/82 [==============================] - 4s 51ms/step - loss: 0.0020 - accuracy: 0.4510 - val_loss: 0.1355 - val_accuracy: 0.4237
Epoch 2/100
82/82 [==============================] - 4s 44ms/step - loss: 0.0020 - accuracy: 0.4511 - val_loss: 0.1634 - val_accuracy: 0.4186
Epoch 3/100
82/82 [==============================] - 4s 45ms/step - loss: 0.0015 - accuracy: 0.4512 - val_loss: 0.1294 - val_accuracy: 0.4242
Epoch 4/100
82/82 [==============================] - 4s 46ms/step - loss: 0.0015 - accuracy: 0.4511 - val_loss: 0.1317 - val_accuracy: 0.4242
Epoch 5/100
82/82 [==============================] - 4s 48ms/step - loss: 0.0016 - accuracy: 0.4512 - val_loss: 0.1351 - val_accuracy: 0.4233
Epoch 6/100
82/82 [==============================] - 4s 48ms/step - loss: 0.0016 - accuracy: 0.4511 - val_loss: 0.1282 - val_accuracy: 0.4256
Epoch 7/100
82/82 [==============================] - 4s 48ms/step - loss: 0.0014 - accuracy: 0.4511 - val_loss: 0.1231 - val_accuracy: 0.4260
Epoch 

In [100]:
#Save the model details.
model.save("rnn_trained")

INFO:tensorflow:Assets written to: rnn_trained\assets


## Model inference

In [101]:
#load the pre-trained model parameters
model = keras.models.load_model("rnn_trained")

#### 1). Encoder inference

In [102]:
encoder_inputs = model.input[0]  
encoder_outputs, state_h_enc, state_c_enc = model.layers[2].output  
encoder_states = [state_h_enc, state_c_enc]
encoder_model = keras.Model(encoder_inputs, encoder_states)

#### 2). Decoder inference

In [103]:
#Decoder model inference
decoder_inputs = model.input[1] 
decoder_state_input_h = keras.Input(shape=(latent_dim,))
decoder_state_input_c = keras.Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_lstm = model.layers[3]
decoder_outputs, state_h_dec, state_c_dec = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h_dec, state_c_dec]
decoder_dense = model.layers[4]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = keras.Model(
    [decoder_inputs] + decoder_states_inputs, [decoder_outputs] + decoder_states)

## Model decoding and text generating

In [104]:
#Define function to translate one-hot vector to to word
reverse_input_char_index = dict((i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict((i, char) for char, i in target_token_index.items())

In [105]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    target_seq = np.zeros((1, 1, num_decoder_tokens))
    target_seq[0, 0, target_token_index["\t"]] = 1.0

    stop_condition = False
    decoded_sentence = ""
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        if sampled_char == "\n" or len(decoded_sentence) > max_decoder_seq_length:
            stop_condition = True

        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.0

        states_value = [h, c]
    return decoded_sentence

In [121]:
for index in range(10):
    # Take one sequence (part of the training set)
    # for trying out decoding.
    input_seq = encoder_input_data[index : index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print("-")
    print("Input sentence:", input_texts[index])
    print("Decoded sentence:", decoded_sentence)

-
Input sentence: apprehensive
Decoded sentence: apprehensiveway

-
Input sentence: desperately
Decoded sentence: esperatelyday

-
Input sentence: grew
Decoded sentence: ewgray

-
Input sentence: niggardly
Decoded sentence: iggardlynay

-
Input sentence: brought
Decoded sentence: oughtbray

-
Input sentence: imagery
Decoded sentence: imageryway

-
Input sentence: quicker
Decoded sentence: ickerquay

-
Input sentence: for
Decoded sentence: orfay

-
Input sentence: emergency
Decoded sentence: emergencyway

-
Input sentence: felt
Decoded sentence: eltfay



In [120]:
print("The final acccuracy:",hist.history['accuracy'][-1])

The final acccuracy: 0.4513489007949829
